In [7]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import ssl

import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split
import torchvision.transforms as transforms

seed = 1234
np.random.seed(seed)

ssl._create_default_https_context = ssl._create_unverified_context

In [8]:
batch_size = 256

gen = torch.Generator()
gen.manual_seed(seed)
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(0, 1)])

trainset, valset = random_split(
    torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform),
    [0.8, 0.2],
    generator=gen
)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset,
    batch_size=batch_size, shuffle=True, num_workers=2, generator=gen)
valloader = torch.utils.data.DataLoader(valset,
    batch_size=batch_size, shuffle=True, num_workers=2, generator=gen)
testloader = torch.utils.data.DataLoader(testset,
    batch_size=batch_size, shuffle=True, num_workers=2, generator=gen)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

print(len(trainset), len(valset), len(testset))

Files already downloaded and verified
40000 10000 10000


In [9]:
class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FeedforwardNeuralNetModel, self).__init__()
        # Linear function
        self.fc1 = nn.Linear(3*input_dim, hidden_dim) 
        # Non-linearity
        self.relu = nn.ReLU()
        # Linear function (readout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)  

    def forward(self, x):
        # Linear function
        x = x.view(-1, 32*32*3)
        out = self.fc1(x)
        # Non-linearity
        out = self.relu(out)
        # Linear function (readout)
        out = self.fc2(out)
        return out

input_dim = 32*32
hidden_dim = 100
output_dim = 10
learning_rate = 0.1
    
model = FeedforwardNeuralNetModel(input_dim, hidden_dim, output_dim)
print(model)

torch.manual_seed(seed)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

FeedforwardNeuralNetModel(
  (fc1): Linear(in_features=3072, out_features=100, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=100, out_features=10, bias=True)
)


In [10]:
iter = 0
for epoch in range(40):
    for i, (images, labels) in enumerate(trainloader):
        # Load images with gradient accumulation capabilities

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        outputs = model(images)

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

        iter += 1

        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in valloader:
                # Load images with gradient accumulation capabilities

                # Forward pass only to get logits/output
                outputs = model(images)

                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)

                # Total number of labels
                total += labels.size(0)

                # Total correct predictions
                correct += (predicted == labels).sum()

            accuracy = 100 * correct / total

            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

Iteration: 500. Loss: 1.6735659837722778. Accuracy: 37.470001220703125
Iteration: 1000. Loss: 1.7399672269821167. Accuracy: 37.650001525878906
Iteration: 1500. Loss: 1.6339384317398071. Accuracy: 44.27000045776367
Iteration: 2000. Loss: 1.4944013357162476. Accuracy: 43.7599983215332
Iteration: 2500. Loss: 1.46949303150177. Accuracy: 43.93000030517578
Iteration: 3000. Loss: 1.610303521156311. Accuracy: 44.279998779296875
Iteration: 3500. Loss: 1.4482430219650269. Accuracy: 45.040000915527344
Iteration: 4000. Loss: 1.4054844379425049. Accuracy: 47.880001068115234
Iteration: 4500. Loss: 1.4764761924743652. Accuracy: 46.290000915527344
Iteration: 5000. Loss: 1.4149024486541748. Accuracy: 46.900001525878906
Iteration: 5500. Loss: 1.3545068502426147. Accuracy: 47.93000030517578
Iteration: 6000. Loss: 1.3688201904296875. Accuracy: 47.83000183105469


In [12]:
correct = 0
total = 0
# Iterate through test dataset
for images, labels in testloader:
    # Forward pass only to get logits/output
    outputs = model(images)

    # Get predictions from the maximum value
    _, predicted = torch.max(outputs.data, 1)

    # Total number of labels
    total += labels.size(0)

    # Total correct predictions
    correct += (predicted == labels).sum()

accuracy = 100 * correct / total

# Print Loss
print('Loss: {}. Accuracy: {}'.format(loss.item(), accuracy))

Loss: 1.4100117683410645. Accuracy: 43.439998626708984
